Open the book in txt format and read it. Then print the total number of characters and the first 100 characters

In [1]:
with open ("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
    
print("Total number of character:", len(raw_text))
print (raw_text[:99])

Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


We need to tokonize the text which means dividing the text into individual words and special charaters.
For the common LLM used by big companies like OpenAI and Meta, they used a massive dataset to train their model but for this one we just use one book for educational purpose and limited resources.

In [2]:
# the "re"- regular expression operator- python liberary used to split the text and build a simple tokenizer
# the following is an example for illustration
import re
text= "Hello, world. This, is a test."
#result= re.split(r'(\s)', text) #\s splitting based on white space
#['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']
result= re.split(r'([, .] |\s)', text) #splitting also , and . you can add all special characters
#['Hello', ', ', 'world', '. ', 'This', ', ', 'is', ' ', 'a', ' ', 'test.']

print (result)

# to get red of the spaces and not consider it as tokens, use .strip() method
#The strip() method removes any leading (starting) and trailing (ending) whitespaces from a given string
new_result= [item for item in result if item.strip()]
print (new_result)

['Hello', ', ', 'world', '. ', 'This', ', ', 'is', ' ', 'a', ' ', 'test.']
['Hello', ', ', 'world', '. ', 'This', ', ', 'is', 'a', 'test.']


In [3]:
# our simple tokenizer to tokenize the entire book
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)',raw_text)

preprocessed= [item for item in preprocessed if item.strip()]
print (preprocessed[:30])
print (len(preprocessed))

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']
4690


# Creating the token IDs

In [18]:
''' 
convert the tokens list into a set, removing all duplicate tokens. Sets only store unique elements.  
then sort the unique tokens alphabetically. 
'''
all_words= sorted(set(preprocessed))
vocab_size= len(all_words)
print(vocab_size) # print the size of unique tokens


#creat a dictionary to have all the unique tokens and their assigned IDs
vocab={token:integer for integer, token in enumerate(all_words)}


1130


In [5]:
#print the first 20 items to see the tokens and its IDs
for i, item in enumerate(vocab.items()):
    print(item)
    if i>=20:
       break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)


In [8]:

class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
                                
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [13]:
text1="""Why _has_ he chucked painting?" I asked abruptly."""
text = """"It's the last he painted, you know," 
           Mrs. Gisburn said with pardonable pride."""

token1=SimpleTokenizerV1(vocab)
ids=token1.encode(text)
print(ids)




[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [14]:
token1.decode(ids)

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

In [15]:
text = "Hello, do you like tea?"
print(token1.encode(text))

KeyError: 'Hello'

The problem is that the word "Hello" was not used in the The Verdict short story. 

Hence, it is not contained in the vocabulary. This highlights the need to consider large and diverse
training sets to extend the vocabulary when working on LLMs.

In [19]:
#ADDING SPECIAL CONTEXT TOKENS supporting two new tokens, <|unk|> for unknown words 
# and <|endoftext|> as an indicator of the end of the first source

all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|unk|>","<|endoftext|>" ])

vocab = {token:integer for integer,token in enumerate(all_tokens)}
len(vocab.items())

1132

In [20]:
#quick check
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|unk|>', 1130)
('<|endoftext|>', 1131)
